# Berger machine translation example

Machine translation example -- English to French -- from the paper 'A
maximum entropy approach to natural language processing' by Berger et
al., 1996.

Consider the translation of the English word 'in' into French.  We
notice in a corpus of parallel texts the following facts:

    (1)    p(dans) + p(en) + p(à) + p(au cours de) + p(pendant) = 1
    (2)    p(dans) + p(en) = 3/10
    (3)    p(dans) + p(à)  = 1/2

This code finds the probability distribution with maximal entropy
subject to these constraints.


In [26]:
import numpy as np
import maxentropy

In [27]:
def f0(x):
    return x in samplespace

def f1(x):
    return x=='dans' or x=='en'

def f2(x):
    return x=='dans' or x=='à'

In [28]:
features = [f0, f1, f2]

samplespace = ['dans', 'en', 'à', 'au cours de', 'pendant']

# Now set the desired feature expectations
target_expectations = [1.0, 0.3, 0.5]

X = np.atleast_2d(target_expectations)

In [4]:
X.shape

(1, 3)

In [5]:
smallmodel = maxentropy.MinDivergenceModel(features, samplespace,
                                           vectorized=False,
                                           verbose=False,
                                           algorithm='BFGS')

In [6]:
smallmodel.features(['en'])

<1x3 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [7]:
# Fit the model
smallmodel.fit(target_expectations)

MinDivergenceModel(algorithm='BFGS',
                   feature_functions=[<function f0 at 0x127b83ec0>,
                                      <function f1 at 0x127b83f60>,
                                      <function f2 at 0x127b8c040>],
                   samplespace=['dans', 'en', 'à', 'au cours de', 'pendant'],
                   verbose=False)

In [8]:
# How well are the constraints satisfied?
assert np.allclose(X[0, :], smallmodel.expectations())

In [9]:
# Manually test if the constraints are satisfied:
p = smallmodel.probdist()
assert np.isclose(p.sum(), target_expectations[0])
assert np.isclose(p[0] + p[1], target_expectations[1])
assert np.isclose(p[0] + p[2], target_expectations[2])

In [10]:
# Output the distribution
print("\nFitted model parameters are:\n" + str(smallmodel.params))
print("\nFitted distribution is:")
for j, x in enumerate(smallmodel.samplespace):
    print(f"\tx = {x:15s}: p(x) = {p[j]:.4f}")


Fitted model parameters are:
[-9.48248421e-16 -5.24869390e-01  4.87527727e-01]

Fitted distribution is:
	x = dans           : p(x) = 0.1859
	x = en             : p(x) = 0.1141
	x = à              : p(x) = 0.3141
	x = au cours de    : p(x) = 0.1929
	x = pendant        : p(x) = 0.1929


In [11]:
# Now show how well the constraints are satisfied:
print()
print("Desired constraints:")
print("\tp['dans'] + p['en'] = 0.3")
print("\tp['dans'] + p['à']  = 0.5")
print()
print("Actual expectations under the fitted model:")
print("\tp['dans'] + p['en'] =", p[0] + p[1])
print("\tp['dans'] + p['à']  =", p[0] + p[2])


Desired constraints:
	p['dans'] + p['en'] = 0.3
	p['dans'] + p['à']  = 0.5

Actual expectations under the fitted model:
	p['dans'] + p['en'] = 0.29999999965207746
	p['dans'] + p['à']  = 0.4999999981007823


## Simulated version

The `maxentropy` package supports simulation for estimating continuous models or discrete ones that are too large to enumerate (e.g. for whole sentences in a natural language).

Here we repeat the above example with simulation. The problem is tiny but we demonstrate how to use simulation conceptually and the API differences when using simulation with `maxentropy` versus with small discrete models.

The following code finds the probability distribution with maximal entropy subject to the same constraints as above **without enumerating the sample space**, using importance sampling instead.

In [12]:
from maxentropy.utils import dictsampler

In [13]:
def f0(x):
    return x in samplespace

def f1(x):
    return x == 'dans' or x == 'en'

def f2(x):
    return x == 'dans' or x == 'à'

features = [f0, f1, f2]

In [14]:
# Define a uniform auxiliary (or "instrumental") distribution for sampling
samplefreq = {e: 1 for e in samplespace}

In [15]:
auxiliary_sampler = dictsampler(samplefreq, size=10**5)

In [16]:
next(auxiliary_sampler)

(array(['en', 'en', 'pendant', ..., 'au cours de', 'dans', 'en'],
       dtype=object),
 array([-1.60943791, -1.60943791, -1.60943791, ..., -1.60943791,
        -1.60943791, -1.60943791]))

In [17]:
model_with_sampling = maxentropy.MCMinDivergenceModel(
    features,
    auxiliary_sampler,
    vectorized=False,
    verbose=False
)

In [18]:
# Now set the desired feature expectations
target_expectations = [1.0, 0.3, 0.5]

X = np.atleast_2d(target_expectations)

In [19]:
model_with_sampling.fit(X)

MCMinDivergenceModel(auxiliary_sampler=<generator object dictsampler at 0x1247c2b20>,
                     feature_functions=[<function f0 at 0x127b8f7e0>,
                                        <function f1 at 0x127b8f240>,
                                        <function f2 at 0x127b8d120>],
                     verbose=False)

In [20]:
# Output the true distribution
print("Fitted model parameters are:")
model_with_sampling.params

Fitted model parameters are:


array([ 8.22449360e-13, -5.27417827e-01,  4.93789531e-01])

We can also use the discrete model fitted above to evaluate how good these fitted parameters are:

In [21]:
smallmodel.setparams(model_with_sampling.params)
p = smallmodel.probdist()

In [22]:
print("\nFitted distribution is:")
for j, x in enumerate(samplespace):
    print(f"\tx = {x:15s}: p(x) = {p[j]:.4f}")


Fitted distribution is:
	x = dans           : p(x) = 0.1861
	x = en             : p(x) = 0.1136
	x = à              : p(x) = 0.3154
	x = au cours de    : p(x) = 0.1925
	x = pendant        : p(x) = 0.1925


In [23]:
# Now show how well the constraints are satisfied:
print()
print("Desired constraints:")
print("\tp['dans'] + p['en'] = 0.3")
print("\tp['dans'] + p['à']  = 0.5")
print()
print("Actual expectations under the fitted model:")
print(f"\tp['dans'] + p['en'] = {p[0] + p[1]:.4f}")
print(f"\tp['dans'] + p['à']  = {p[0] + p[2]:.4f}")


Desired constraints:
	p['dans'] + p['en'] = 0.3
	p['dans'] + p['à']  = 0.5

Actual expectations under the fitted model:
	p['dans'] + p['en'] = 0.2997
	p['dans'] + p['à']  = 0.5015


In [25]:
print("\nEstimated error in constraint satisfaction (should be close to 0):\n",
        abs(model_with_sampling.feature_expectations() - X))
print("\nTrue error in constraint satisfaction (should be close to 0):\n",
        abs(smallmodel.feature_expectations() - X))


Estimated error in constraint satisfaction (should be close to 0):
 [[2.44471110e-13 5.43037115e-10 9.18609133e-10]]

True error in constraint satisfaction (should be close to 0):
 [[0.         0.0003106  0.00147411]]
